# Import Library

In [ ]:
import tensorflow as tf
import keras
import numpy as np
from sklearn.model_selection import train_test_split
import os
import matplotlib.pyplot as plt

# Load Dataset

In [ ]:
img_rows, img_cols = 224, 224

data_path = "./Dataset/"

x, y = [], []

counter = 0

classes = os.listdir(data_path)

for class_path in classes:
    for img_path in os.listdir (f"{data_path}{class_path}"):
        
        path = f"{data_path}{class_path}/{img_path}"
        label = counter
        image = tf.keras.preprocessing.image.load_img(path, target_size=(img_rows, img_cols), color_mode="grayscale")
        image = tf.keras.preprocessing.image.img_to_array(image)
        
        x.append(image)
        y.append(label)
        
    counter += 1
    
x=np.array(x)
y=np.array(y)

In [ ]:
# normalisasi
x = x.astype("float32") / 255.0

# split 80:10:10
X_train, X_temp, y_train, y_temp = train_test_split(x, y, test_size=0.2, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

# one hot encoding
y_train = keras.utils.to_categorical(y_train, num_classes=4)
y_val = keras.utils.to_categorical(y_val, num_classes=4)
y_test = keras.utils.to_categorical(y_test, num_classes=4)

# Modelling

In [ ]:
#model
model = keras.Sequential()
model.add(keras.layers.Conv2D(6, (5,5), activation="relu", input_shape=(img_rows, img_cols, 1)))
#shape 1 krn gray scale, sesuaiin sama warna gambar
model.add(keras.layers.MaxPooling2D(pool_size=(2,2), strides=2))
model.add(keras.layers.Conv2D(16, (5, 5), activation="relu"))
model.add(keras.layers.Conv2D(16, (5,5), activation="relu"))
model.add(keras.layers.MaxPooling2D(pool_size=(2,2), strides=2))
model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(120, activation='relu'))
model.add(keras.layers.Dense(4, activation='softmax')) #sesuaiin dgn jumlah class kalian
model.summary()

In [ ]:
from tensorflow.keras.utils import plot_model

plot_model(model, to_file='model.png', show_shapes=True, show_layer_names=True)

# Training Modelling

In [ ]:
model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=['accuracy'])
history=model.fit(X_train, y_train, batch_size=120, validation_data=(X_val, y_val), epochs=10)

# Evaluate

In [ ]:
score = model.evaluate(X_test, y_test)
print(f"test loss: {score[0]}")
print(f"test accuracy: {score[1]}")

# Predict

In [ ]:
predictions = model.predict(X_test)
true_labels = np.argmax(y_test, axis=1)
predict_label = np.argmax(predictions, axis=1)

for i in range(9):
    plt.subplot(3, 3, i+1)
    plt.imshow(X_test[i], cmap='gray')
    plt.axis('off')
    plt.text(0.5, -0.15, f"Pred: {classes[predict_label[i]]}", transform=plt.gca().transAxes)
    plt.text(0.5, -0.3, f"True: {classes[true_labels[i]]}", transform=plt.gca().transAxes)
    
plt.tight_layout() #biar textnya yg true ga nutup foto dibawahnya
plt.show()

# Plotting

In [ ]:
plt.figure(figsize=(10,6))

plt.subplot(1,2,1)
plt.plot(history.history['accuracy'], label="Training Accuracy")
plt.plot(history.history['val_accuracy'], label="Validation Accuracy")

plt.xlabel("Number of Epochs")
plt.ylabel("Accuracy [%]")
plt.title("Accuracy")

plt.legend()

plt.subplot(1,2,2)
plt.plot(history.history['loss'], label="Training Loss")
plt.plot(history.history['val_loss'], label="Validation Loss")

plt.xlabel("Number of Epochs")
plt.ylabel("Loss [%]")
plt.title("Loss")

plt.legend()